<a href="https://colab.research.google.com/github/Lsyyy623/Case-Competition-Pillar-1-Team-11/blob/main/ucb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import time
from urllib.parse import urljoin

import pandas as pd
import requests
from bs4 import BeautifulSoup

BASE = "https://haas.berkeley.edu"
SEARCH_URL = BASE + "/faculty/?action=faculty-search&letter={letter}"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}


def get_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def collect_all_profile_links() -> list[str]:
    """从 A-Z 的搜索结果页收集所有教授个人主页链接（去重）。"""
    links = set()

    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        url = SEARCH_URL.format(letter=letter)
        soup = get_soup(url)

        for a in soup.select("a[href]"):
            href = a.get("href", "").strip()
            if not href:
                continue

            full = urljoin(BASE, href)

            # 只保留形如 https://haas.berkeley.edu/faculty/<slug>/
            if re.match(r"^https://haas\.berkeley\.edu/faculty/[^/]+/?$", full):
                # 排除目录页本身
                if full.rstrip("/") == (BASE + "/faculty").rstrip("/"):
                    continue
                links.add(full.rstrip("/") + "/")

        time.sleep(0.4)  # 友好一点，别太快

    return sorted(links)


def extract_selected_papers(profile_url: str) -> tuple[str, list[str]]:
    """从个人主页抓取 Selected Papers and Publications 的条目文本列表。"""
    soup = get_soup(profile_url)

    # 姓名：通常在 h1
    h1 = soup.find("h1")
    name = h1.get_text(" ", strip=True) if h1 else ""

    # 找到“Selected Papers and Publications”标题
    header = None
    for tag in soup.find_all(["h2", "h3", "h4"]):
        t = tag.get_text(" ", strip=True)
        if "Selected Papers and Publications" in t:
            header = tag
            break

    if not header:
        return name, []

    entries = []

    # 从 header 往后扫：直到遇到下一个标题（h2/h3/h4）就停止
    # 优先抓 li；如果没有 li，就抓 p（有些页面会写“Please visit my website...”）
    for elem in header.find_all_next():
        if elem == header:
            continue
        if elem.name in ["h2", "h3", "h4"]:
            break

        if elem.name == "li":
            txt = " ".join(elem.stripped_strings)
            if txt:
                entries.append(txt)
        elif elem.name == "p" and not entries:
            txt = " ".join(elem.stripped_strings)
            if txt:
                entries.append(txt)

    # 清理/去重
    cleaned = []
    seen = set()
    for t in entries:
        t = re.sub(r"\s+", " ", t).strip()
        if t and t not in seen:
            cleaned.append(t)
            seen.add(t)

    return name, cleaned


def extract_year(entry: str):
    """从条目里提取年份：取最后一个 19xx/20xx。没有就空。"""
    years = re.findall(r"(?:19|20)\d{2}", entry)
    return int(years[-1]) if years else pd.NA


def main():
    profile_links = collect_all_profile_links()
    print(f"Found {len(profile_links)} profile pages.")

    rows = []
    for i, url in enumerate(profile_links, 1):
        print(f"[{i}/{len(profile_links)}] {url}")
        try:
            name, entries = extract_selected_papers(url)
            for entry in entries:
                rows.append(
                    {
                        "name": name,
                        "profile link": url,
                        "entry": entry,
                        "year": extract_year(entry),
                    }
                )
        except Exception as e:
            print("  ERROR:", e)

        time.sleep(0.6)  # 控制频率，减少被封风险

    df = pd.DataFrame(rows, columns=["name", "profile link", "entry", "year"])
    df["year"] = df["year"].astype("Int64")  # 允许空值的整型
    out = "haas_selected_papers.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")
    print(f"Saved: {out}  (rows={len(df)})")


if __name__ == "__main__":
    main()


Found 359 profile pages.
[1/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=A/
[2/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=B/
[3/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=C/
[4/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=D/
[5/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=E/
[6/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=F/
[7/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=G/
[8/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=H/
[9/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=I/
[10/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=J/
[11/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=K/
[12/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter=L/
[13/359] https://haas.berkeley.edu/faculty/?action=faculty-search&letter